# High-level parcels structure tutorial

There are many different ways in which to use Parcels for research. Many different configurations, multiple files in different formats, kernels written for specific field configurations or specific particle configurations and kernels that only work in combination with other kernels. The flexibility of the parcels code enables this wide range of applicability, but can also make your code messy which increases the chance of bugs. For a smooth programming experience with Parcels, it is recommended you make a general structure in setting up your simulations. A good practice is to separate the different parts of your code into sections:
1. [**FieldSet**](#FieldSet). Load and set up the (velocity) fields that your particles need to access
2. [**ParticleSet**](#ParticleSet). Define the type of particles you want to release, what `Variables` they have and what their initial conditions are
3. [**Execute kernels**](#Kernel-execution). Define and compile the kernels that say what your particles need to do each timestep and execute them
4. [**Output**](#Output). Write and store the output to a netcdf file. What you can do with the output in terms of analysis is documented [**here**](https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_output.ipynb)

The [parcels tutorial](https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/parcels_tutorial.ipynb) shows a very basic example of this setup, altough it is spread over multiple cells with text in between:

In [4]:
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4

# 1. Setting up the velocity fields in a FieldSet object

fieldset = FieldSet.from_parcels("MovingEddies_data/moving_eddies")


# 2. Defining the particles type and initial conditions in a ParticleSet object

pset = ParticleSet.from_list(fieldset=fieldset,   # the fields on which the particles are advected
                             pclass=JITParticle,  # the type of particles (JITParticle or ScipyParticle)
                             lon=[3.3e5,  3.3e5], # a vector of release longitudes 
                             lat=[1e5, 2.8e5])    # a vector of release latitudes


# 3. Executing an advection kernel on the given fieldset

output_file = pset.ParticleFile(name="EddyParticles.nc", outputdt=3600) # the file name and the time step of the outputs
pset.execute(AdvectionRK4,                 # the kernel (which defines how particles move)
             runtime=86400*6,              # the total length of the run
             dt=300,                       # the timestep of the kernel
             output_file=output_file)


# 4. Exporting the simulation output to a netcdf file

output_file.export()

INFO: Compiled JITParticleAdvectionRK4 ==> C:\Users\GEBRUI~1\AppData\Local\Temp\parcels-tmp\defb7bc275d58865cd007ea8f826f558_0.dll


When you start making the parcels simulation more complex, it is a good idea to keep these different steps separate to keep a clear overview and find bugs more easily

# FieldSet

Parcels provides a framework to simulate the movement of particles **within an existing flowfield environment**. To start a parcels simulation we must define this environment with the [**`FieldSet`** class](https://oceanparcels.org/gh-pages/html/#module-parcels.fieldset). The minimal requirements for this Fieldset are that it must contain the `'U'` and `'V'` fields: the 2D hydrodynamic data that will move the particles. 

The general method to use is [**`FieldSet.from_netcdf`**](https://oceanparcels.org/gh-pages/html/#parcels.fieldset.FieldSet.from_netcdf), which requires `filenames`, `variables` and `dimensions`. Each of these is a dictionary, and `variables` requires at least a `U` and `V`, but any other `variable` can be added too (e.g. temperature, mixedlayerdepth, etc). Note also that `filenames` can contain wildcards. For example, the GlobCurrent data that is shipped with Parcels can be read with: 

In [6]:
fname = 'GlobCurrent_example_data/*.nc'
filenames = {'U': fname, 'V': fname}
variables = {'U': 'eastward_eulerian_current_velocity', 'V': 'northward_eulerian_current_velocity'}
dimensions = {'lat': 'lat', 'lon': 'lon', 'time': 'time'}
fset = FieldSet.from_netcdf(filenames, variables, dimensions)

Note that `dimensions` can also be a dictionary-of-dictionaries. For example, if you have wind data on a completely different grid, you can do:

In [ ]:
fname = 'GlobCurrent_example_data/*.nc'
wname = 'path_to_your_windfiles'
filenames = {'U': fname, 'V': fname, 'wind': wname}
variables = {'U': 'eastward_eulerian_current_velocity', 'V': 'northward_eulerian_current_velocity', 'wind': 'wind'}
dimensions = {}
dimensions['U'] = {'lat': 'lat', 'lon': 'lon', 'depth': 'depth', 'time': 'time'}
dimensions['V'] = {'lat': 'lat', 'lon': 'lon', 'depth': 'depth', 'time': 'time'}
dimensions['wind'] = {'lat': 'latw', 'lon': 'lonw', 'time': 'time'}
fset = FieldSet.from_netcdf(filenames, variables, dimensions)

In a similar way, you can add `U` and `V` fields that are on different grids (e.g. Arakawa C-grids). Parcels will take care under the hood that the different grids are dealt with properly.

# ParticleSet

Once you have set up the environment with the FieldSet object, you can start defining your particles in a [**`ParticleSet`** object](https://oceanparcels.org/gh-pages/html/#module-parcels.particleset). This object requires the [**`FieldSet`**](https://oceanparcels.org/gh-pages/html/#module-parcels.fieldset) in which it exists, the type of [**`Particle`**](https://oceanparcels.org/gh-pages/html/#module-parcels.particle) which contains the information each particle will store and the initial conditions for each [**`Variable`**](https://oceanparcels.org/gh-pages/html/#parcels.particle.Variable) defined in the `Particle`, most notably the release locations in `lon` and `lat`.

In [9]:
pset = ParticleSet(fieldset=fieldset,   # the fields on which the particles are advected
                   pclass=JITParticle,  # the type of particles (JITParticle or ScipyParticle)
                   lon=3.3e5,           # release longitude
                   lat=1e5)             # release latitude

The different [**`Particle`**](https://oceanparcels.org/gh-pages/html/#module-parcels.particle) types available are the [**`JITParticle`**](https://oceanparcels.org/gh-pages/html/#parcels.particle.JITParticle) and the [**`ScipyParticle`**](https://oceanparcels.org/gh-pages/html/#parcels.particle.ScipyParticle), but it is very easy to create your own particle class which includes other [**`Variables`**](https://oceanparcels.org/gh-pages/html/#parcels.particle.Variable):

In [13]:
from parcels import Variable

class PressureParticle(JITParticle):         # Define a new particle class
    p = Variable('p', initial=0)             # Variable 'p' with initial value 0.

# Kernel execution

After defining the environment with `FieldSet` and the particle information with `ParticleSet`, we can move on to actually running the parcels simulation by using [**`ParticleSet.execute()`**](https://oceanparcels.org/gh-pages/html/#parcels.particleset.ParticleSet.execute). 

In [14]:
output_file = pset.ParticleFile(name="EddyParticles.nc", outputdt=3600) # the file name and the time step of the outputs
pset.execute(AdvectionRK4,                 # the kernel (which defines how particles move)
             runtime=86400*6,              # the total length of the run
             dt=300,                       # the timestep of the kernel
             output_file=output_file)

INFO: Compiled JITParticleAdvectionRK4 ==> C:\Users\GEBRUI~1\AppData\Local\Temp\parcels-tmp\fe0899895db2ee20d980323f5e2c6191_0.dll


One of the most powerful features of Parcels is the ability to write custom Kernels (see e.g. [this example](http://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/parcels_tutorial.ipynb#Adding-a-custom-behaviour-kernel) in the parcels tutorial). These Kernels are little snippets of code that get executed by Parcels, giving the ability to add 'behaviour' to particles.

However, there are some key limitations to the Kernels that everyone who wants to write their own should be aware of:
* Every Kernel must be a function with the following (and only those) arguments: `(particle, fieldset, time)`
* In order to run successfully in JIT mode, Kernel definitions can only contain the following types of commands:

# Output

While executing the `ParticleSet`, parcels stores the data in **npy** files in an output folder. To take all the data and store them in a netcdf file, you can use [**`ParticleFile.export()`**](https://oceanparcels.org/gh-pages/html/#parcels.particlefile.ParticleFile.export) if you want to keep the folder with npy files, or [**`ParticleFile.close()`**](https://oceanparcels.org/gh-pages/html/#parcels.particlefile.ParticleFile.close) if you only want to keep the netcdf file:

In [ ]:
output_file.export()
output_file.close()